**SCENIC分析**

成年 胚胎统一输入，并做pseudobulk进行均衡。

In [1]:
library(Seurat)
library(tidyverse)

Attaching SeuratObject

Warning message in system("timedatectl", intern = TRUE):
“running command 'timedatectl' had status 1”
── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.2     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


In [2]:
obj_adult = readRDS("/data/input/Files/FB_data/scRNA_FB_20240703/Adult_FB_fascia_myo_trajectory.rds")
meta_adult = read.csv("/data/input/Files/FB_data/scRNA_FB_20240703/adult_myo_monocle3_meta.csv.gz", row.names=1)
obj_adult@meta.data[,colnames(meta_adult)] = meta_adult

x = c()
tb = table(obj_adult$celltype_240516)
n = 0
nbin = 100
for (i in c("FB_fascia_1_Adult", "FB_inflammatory_Adult", "FB_oxidative_stress_Adult", "FB_myo_Adult")){
    x = c(x, ntile(1:tb[[i]], nbin) + nbin * n)
    n = n + 1
    }
obj_adult$group = x[rank(obj_adult$pseudotime)]

In [3]:
obj_ep = readRDS("/data/input/Files/FB_data/scRNA_FB_20240703/EP_FB_fascia_myo_trajectory.rds")
meta_ep = read.csv("/data/input/Files/FB_data/scRNA_FB_20240703/ep_myo_monocle3_meta.csv.gz", row.names=1)
obj_ep@meta.data[,colnames(meta_ep)] = meta_ep

x = c()
tb = table(obj_ep$celltype_0519)
n = 0
nbin = 100
for (i in c("FB_fascia_EP", "FB_inflammatory_EP", "FB_oxidative_stress_EP")){
    x = c(x, ntile(1:tb[[i]], nbin) + nbin * n)
    n = n + 1
    }
obj_ep$group = x[rank(obj_ep$pseudotime)]

In [4]:
df = read.csv("/data/input/Files/FB_data/scRNA_FB_20240703/Myo_trac_EP_All_240526_trajectory_genes_MI_05.csv")
genes = df$gene_short_name
df = read.csv("/data/input/Files/FB_data/scRNA_FB_20240703/Myo_trac_Adult_All_240526_trajectory_genes_MI_05.csv")
genes = c(genes,df$gene_short_name)

genes = unique(genes)

In [5]:
mat_adult = AverageExpression(obj_adult, group.by = "group", features=genes)$RNA
mat_ep = AverageExpression(obj_ep, group.by = "group", features=genes)$RNA
mat = cbind(mat_adult, mat_ep)


dir.create("SCENIC_240807")
write.csv(t(mat),file = gzfile('SCENIC_240807/myo_trace_genes_count.csv.gz'),quote=F)

**python** 

构建loom

In [1]:
import loompy as lp
import numpy as np
import scanpy as sc

Matplotlib created a temporary config/cache directory at /tmp/matplotlib-7q7_bm0r because the default path (/home/stereonote/.config/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


In [2]:
output="SCENIC_240807/All.loom"
input="SCENIC_240807/myo_trace_genes_count.csv.gz"

x=sc.read_csv(input)
row_attrs = {"Gene": np.array(x.var_names),}
col_attrs = {"CellID": np.array(x.obs_names)}
lp.create(output, x.X.transpose(), row_attrs, col_attrs)


**pyscenic**

In [3]:
%%bash
#default value
input_loom=/data/work/fig3_TF/SCENIC_240807/All.loom
n_workers=15
output_path="/data/work/fig3_TF/SCENIC_240807/"

database='/data/input/Files/SCENIC/00.databases/01.mouse'
tfs=${database}/allTFs_mm.txt
feather=${database}/mm10_10kbp_up_10kbp_down_full_tx_v10_clust.genes_vs_motifs.rankings.feather
tbl=${database}/motifs-v10nr_clust-nr.mgi-m0.001-o0.0.tbl
pyscenic=pyscenic

# grn
$pyscenic grn \
--num_workers $n_workers \
--output ${output_path}/grn.tsv \
--method grnboost2 \
$input_loom  $tfs

Numba: Attempted to fork from a non-main thread, the TBB library may be in an invalid state in the child process.

2024-08-07 10:18:33,832 - pyscenic.cli.pyscenic - INFO - Loading expression matrix.

2024-08-07 10:18:33,978 - pyscenic.cli.pyscenic - INFO - Inferring regulatory networks.
Numba: Attempted to fork from a non-main thread, the TBB library may be in an invalid state in the child process.
Numba: Attempted to fork from a non-main thread, the TBB library may be in an invalid state in the child process.
Numba: Attempted to fork from a non-main thread, the TBB library may be in an invalid state in the child process.
Numba: Attempted to fork from a non-main thread, the TBB library may be in an invalid state in the child process.
Numba: Attempted to fork from a non-main thread, the TBB library may be in an invalid state in the child process.
Numba: Attempted to fork from a non-main thread, the TBB library may be in an invalid state in the child process.
Numba: Attempted to fork fro

preparing dask client
parsing input
creating dask graph
15 partitions
computing dask graph
not shutting down client, client was created externally
finished



2024-08-07 10:25:48,689 - pyscenic.cli.pyscenic - INFO - Writing results to file.


In [ ]:
%%bash
#default value
input_loom=/data/work/fig3_TF/SCENIC_240807/All.loom
n_workers=30
output_path="/data/work/fig3_TF/SCENIC_240807/"

database='/data/input/Files/SCENIC/00.databases/01.mouse'
tfs=${database}/allTFs_mm.txt
feather=${database}/mm10_10kbp_up_10kbp_down_full_tx_v10_clust.genes_vs_motifs.rankings.feather
tbl=${database}/motifs-v10nr_clust-nr.mgi-m0.001-o0.0.tbl
pyscenic=pyscenic

# cistarget
$pyscenic ctx \
${output_path}/grn.tsv $feather \
--annotations_fname $tbl \
--expression_mtx_fname $input_loom \
--mode "dask_multiprocessing" \
--output ${output_path}/ctx.csv \
--num_workers $n_workers   \
--mask_dropouts

Numba: Attempted to fork from a non-main thread, the TBB library may be in an invalid state in the child process.

2024-08-07 10:28:08,304 - pyscenic.cli.pyscenic - INFO - Creating modules.

2024-08-07 10:28:08,967 - pyscenic.cli.pyscenic - INFO - Loading expression matrix.

2024-08-07 10:28:09,333 - pyscenic.utils - INFO - Calculating Pearson correlations.

2024-08-07 10:28:09,457 - pyscenic.utils - WARNING - Note on correlation calculation: the default behaviour for calculating the correlations has changed after pySCENIC verion 0.9.16. Previously, the default was to calculate the correlation between a TF and target gene using only cells with non-zero expression values (mask_dropouts=True). The current default is now to use all cells to match the behavior of the R verision of SCENIC. The original settings can be retained by setting 'rho_mask_dropouts=True' in the modules_from_adjacencies function, or '--mask_dropouts' from the CLI.
	Dropout masking is currently set to [True].

2024-08

In [ ]:
# AUCell
#default value
input_loom=/data/work/fig3_TF/SCENIC_240807/All.loom
n_workers=15
output_path="/data/work/fig3_TF/SCENIC_240807/"

database='/data/input/Files/SCENIC/00.databases/01.mouse'
tfs=${database}/allTFs_mm.txt
feather=${database}/mm10_10kbp_up_10kbp_down_full_tx_v10_clust.genes_vs_motifs.rankings.feather
tbl=${database}/motifs-v10nr_clust-nr.mgi-m0.001-o0.0.tbl
pyscenic=pyscenic

$pyscenic aucell \
$input_loom \
${output_path}/ctx.csv \
--output ${output_path}/aucell.loom \
--num_workers $n_workers


In [ ]:
input_loom=/data/work/fig3_TF/SCENIC_240806/All.loom
n_workers=15
output_path="/data/work/fig3_TF/SCENIC_240806/"

database='/data/input/Files/SCENIC/00.databases/01.mouse'
tfs=${database}/allTFs_mm.txt
feather=${database}/mm10_10kbp_up_10kbp_down_full_tx_v10_clust.genes_vs_motifs.rankings.feather
tbl=${database}/motifs-v10nr_clust-nr.mgi-m0.001-o0.0.tbl
pyscenic=pyscenic

$pyscenic aucell \
$input_loom \
${output_path}/ctx.csv \
--output ${output_path}/aucell_auc0.01.loom \
--num_workers $n_workers \
--auc_threshold 0.01

$pyscenic aucell \
$input_loom \
${output_path}/ctx.csv \
--output ${output_path}/aucell_nes2.loom \
--num_workers $n_workers \
--nes_threshold 2.0